In [1]:
import warnings
warnings.filterwarnings('ignore')

from IPython.display import Image, display, HTML, display_html
import ipywidgets
import pickle
import glob
import os
import datetime
import dask.dataframe as dd
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# from ggplot import *
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline
plt.style.use('fivethirtyeight')
from matplotlib.ticker import PercentFormatter
#plotly
import plotly
# import plotly.plotly as py
import plotly.offline as pyo
import plotly.figure_factory as ff
from   plotly.tools import FigureFactory as FF, make_subplots
import plotly.graph_objs as go

import plotly.io as pio
from   plotly import tools
from   plotly.offline import download_plotlyjs, init_notebook_mode, iplot
init_notebook_mode(connected=True)

import plotly.express as px
## SPlunk Related 

import pyodbc

#Bokeh

# Import figure from bokeh.plotting
from bokeh.plotting import figure
from bokeh.io import output_file, show, output_notebook
from bokeh.models import CategoricalColorMapper

In [2]:
df = sns.load_dataset("mpg")

In [3]:
df.shape

(398, 9)

In [4]:
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino


In [5]:
df.cylinders.unique()

array([8, 4, 6, 3, 5], dtype=int64)

In [6]:
df.cylinders.value_counts().to_frame().sort_index()

,cylinders
3,4
4,204
5,3
6,84
8,103


In [7]:
count_per_cylinders = df.cylinders.value_counts().to_frame().sort_index().reset_index()
count_per_cylinders.columns = ['cylinders', 'Count']
px.bar(count_per_cylinders, x =  'cylinders', y = 'Count', hover_data=['Count'], title = '# of Cars per Cylinder type')

In [8]:
def ploltyl_ex_scatter(df, x, y):
    
#     colrmap = {1 : 'green' , 2 : 'orange', 3 : 'red'}
    fig = px.scatter(df, x=x, y=y, color='cylinders', size='cylinders',color_discrete_map = 'coolwarm',width = 1000, height = 600)
    fig.show()

In [9]:
df.columns

Index(['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model_year', 'origin', 'name'], dtype='object')

In [10]:
x = df.columns[3]
y = df.columns[2]
fig = px.scatter(df, x=x, y=y, title=f'{x} vs. {y}')
fig.show()

In [11]:
x = df.columns[3]
y = df.columns[2]
fig = px.scatter(df, x=x, y=y, color='cylinders', size='cylinders',color_continuous_scale=px.colors.sequential.Rainbow,opacity=0.5,
                 title=f'{x} vs. {y}')
fig.show()

# Subplots

In [12]:
df.origin.unique().tolist()

['usa', 'japan', 'europe']

In [13]:

x = df.columns[3]
y = df.columns[2]
fig = px.scatter(df, x=x, y=y, color='cylinders', size='cylinders',facet_col="origin",
                 color_continuous_scale=px.colors.sequential.Rainbow,opacity=0.5,
                 width = 1000, height = 500)
fig.layout.title = f'Scatter plot of {y} and {x} by Origin'
fig.layout.font = dict(size = 20)
fig.show()

In [14]:
cols = df.select_dtypes(include=['float', 'int']).columns.tolist()
fig = px.scatter_matrix(df, dimensions= cols, color = 'cylinders', color_continuous_scale=px.colors.sequential.Rainbow)
fig.show()

In [15]:

df_count_car = df.origin.value_counts().to_frame().reset_index()
df_count_car.columns = ['Origin', 'Count']
f = px.bar(df_count_car,x = 'Origin', y = 'Count')
f.show()

# Box Plot

In [16]:
x = df.columns[3]
y = df.columns[2]
f = px.box(df, x='origin', y = y)
f.show()

# Dash and Plotly Express

In [17]:
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

In [18]:
df = sns.load_dataset("mpg")

# Dash App

In [ ]:

app = dash.Dash()
app.layout = html.Div([
    html.H1('MPG Data with Plotly Express'),
    
    dcc.RadioItems(id = 'selectplot',
                 options = [dict(label = 'bar'     , value = 'bar'),
                            dict(label = 'scatter' , value = 'scatter'),
                            dict(label = 'scat_mat', value = 'scat_mat')
                           ], 
                 value  = 'bar'
                          ),
    
    dcc.Graph(id = 'plot', figure = {}, style={"width": "75%", "display": "inline-block"})
     
])

@app.callback(Output('plot', 'figure'),
              [Input('selectplot', 'value')])
def displayplot(value):
#     print(value)
    
    if value == 'bar':
        df_count_car = df.origin.value_counts().to_frame().reset_index()
        df_count_car.columns = ['Origin', 'Count']
        fig = px.bar(df_count_car,x = 'Origin', y = 'Count')
        return fig
    elif value == 'scatter':
        x = df.columns[3]
        y = df.columns[2]
        fig = px.scatter(df, x=x, y=y, color='cylinders', size='cylinders',facet_col="origin",
                 color_continuous_scale=px.colors.sequential.Rainbow,opacity=0.5,
                 width = 1000, height = 500)
        return fig
    else:
        cols = df.select_dtypes(include=['float', 'int']).columns.tolist()
        fig = px.scatter_matrix(df, dimensions= cols, color = 'cylinders', color_continuous_scale=px.colors.sequential.Rainbow)
        return fig
if __name__ == '__main__':
    app.run_server(port = 8099)
    
 



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8099/ (Press CTRL+C to quit)
127.0.0.1 - - [16/Oct/2019 09:44:00] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Oct/2019 09:44:00] "GET /_dash-component-suites/dash_renderer/react@16.8.6.min.js?v=1.0.0&m=1566551547 HTTP/1.1" 200 -
127.0.0.1 - - [16/Oct/2019 09:44:00] "GET /_dash-component-suites/dash_renderer/react-dom@16.8.6.min.js?v=1.0.0&m=1566551547 HTTP/1.1" 200 -
127.0.0.1 - - [16/Oct/2019 09:44:00] "GET /_dash-component-suites/dash_renderer/prop-types@15.7.2.min.js?v=1.0.0&m=1566551547 HTTP/1.1" 200 -
127.0.0.1 - - [16/Oct/2019 09:44:00] "GET /_dash-component-suites/dash_html_components/dash_html_components.min.js?v=1.0.0&m=1566551552 HTTP/1.1" 200 -
127.0.0.1 - - [16/Oct/2019 09:44:00] "GET /_dash-component-suites/dash_core_components/highlight.pack.js?v=1.1.1&m=1566551549 HTTP/1.1" 200 -
127.0.0.1 - - [16/Oct/2019 09:44:00] "GET /_dash-component-suites/dash_renderer/dash_renderer.min.js?v=1.0.0&m=1566551547 HTTP/1.1" 200 -
127.0.0.1 - - [16/Oct/2019 0